In [1]:
%run ../src/parsing/preset.py

In [2]:
model = open_model("XGBoost", "../models/")

In [3]:
forecast_data = create_forecast_data(1, 'm', "../data/feature_data.gzip")

In [4]:
forecast_data.sort_index(axis=1, inplace=True)
forecast_data.sort_values(by=["datetime", "subject_name"], inplace=True)
forecast_data.drop_duplicates(subset=["datetime", "subject_name"], inplace=True)

In [5]:
forecast_data

,actual_consumption,datetime,subject_name
0,NaN,2024-08-18 12:00:00,Алтайский край
1,NaN,2024-08-18 12:00:00,Амурская область
2,NaN,2024-08-18 12:00:00,Архангельская область
3,NaN,2024-08-18 12:00:00,Астраханская область
4,NaN,2024-08-18 12:00:00,Белгородская область
...,...,...,...
53995,NaN,2024-09-17 11:00:00,Челябинская область
53996,NaN,2024-09-17 11:00:00,Чеченская Республика
53997,NaN,2024-09-17 11:00:00,Чувашская Республика - Чувашия
53998,NaN,2024-09-17 11:00:00,Южно-Якутский энергорайон


In [6]:
main_df = pd.read_parquet("../data/feature_data.gzip")

In [7]:
main_df["subject_name"] = decoding(main_df[uniq_subjs])

In [8]:
main_df[main_df["Ярославская область"] == 1]

,actual_consumption,time_dummie,year,month,day_of_month,hour,day_of_week,day_of_year,week_of_year,quarter,...,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область,subject_name
70,1013.0,8783,2012,2,20,0,1,51,8,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
141,977.0,8784,2012,2,20,1,1,51,8,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
212,978.0,8785,2012,2,20,2,1,51,8,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
283,986.0,8786,2012,2,20,3,1,51,8,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
354,1014.0,8787,2012,2,20,4,1,51,8,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5646967,758.0,116581,2024,8,18,7,7,231,33,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
5647042,831.0,116582,2024,8,18,8,7,231,33,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
5647117,842.0,116583,2024,8,18,9,7,231,33,3,...,0,0,0,0,0,0,0,0,1,Ярославская область
5647192,864.0,116584,2024,8,18,10,7,231,33,3,...,0,0,0,0,0,0,0,0,1,Ярославская область


In [9]:
main_df.sort_index(axis=1, inplace=True)
# main_df.sort_values(by=["year", "month", "day_of_year", "day_of_week", "hour", "subject_name"], inplace=True)
# main_df.drop_duplicates(subset=["year", "month", "day_of_year", "day_of_week", "hour", "subject_name"], inplace=True)

In [10]:
forecast_data_rows_count = forecast_data.shape[0]
main_df_rows_count = main_df.shape[0]

In [11]:
forecast_data["datetime"] = forecast_data["datetime"].astype(str)
forecast_data = create_time_step_features(forecast_data, max(main_df["time_dummie"]))  # Check function
forecast_data = pd.concat([forecast_data, pd.get_dummies(forecast_data["subject_name"], columns=uniq_subjs)], axis=1)
forecast_data["lag_hour"] = np.array([np.nan] * forecast_data_rows_count)
forecast_data["lag_day"] = np.array([np.nan] * forecast_data_rows_count)
forecast_data["lag_week"] = np.array([np.nan] * forecast_data_rows_count)
forecast_data["lag_month"] = np.array([np.nan] * forecast_data_rows_count)
forecast_data["lag_year"] = np.array([np.nan] * forecast_data_rows_count)

In [12]:
forecast_data.drop(columns=["datetime"], inplace=True)

In [13]:
forecast_data.sort_index(axis=1, inplace=True)
forecast_data.sort_values(by=["year", "month", "day_of_year", "day_of_week", "hour", "subject_name"], inplace=True)
forecast_data.drop_duplicates(subset=["year", "month", "day_of_year", "day_of_week", "hour", "subject_name"], inplace=True)

In [14]:
##############

main_df = pd.concat(
                    [main_df, forecast_data[:100]],
                    axis=0,
                    ignore_index=True
                   )

In [15]:
main_df.index += 1

In [16]:
main_df

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
1,1302.0,20,1,51,0,0,1284.0,1339.0,1367.0,1361.0,...,0,0,0,0,0,0,0,0,0,0
2,1091.0,20,1,51,0,0,1001.0,1034.0,1021.0,1116.0,...,0,0,0,0,0,0,0,0,0,0
3,946.0,20,1,51,0,0,959.0,982.0,926.0,996.0,...,0,0,0,0,0,0,0,0,0,0
4,615.0,20,1,51,0,0,617.0,638.0,535.0,644.0,...,0,0,0,0,0,0,0,0,0,0
5,1679.0,20,1,51,0,0,1684.0,1803.0,1712.0,1714.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647364,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647365,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647366,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647367,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [17]:
main_df.drop_duplicates(subset=["year", "month", "day_of_year", "day_of_week", "hour", "subject_name"], inplace=True)

In [18]:
main_df

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
1,1302.0,20,1,51,0,0,1284.0,1339.0,1367.0,1361.0,...,0,0,0,0,0,0,0,0,0,0
2,1091.0,20,1,51,0,0,1001.0,1034.0,1021.0,1116.0,...,0,0,0,0,0,0,0,0,0,0
3,946.0,20,1,51,0,0,959.0,982.0,926.0,996.0,...,0,0,0,0,0,0,0,0,0,0
4,615.0,20,1,51,0,0,617.0,638.0,535.0,644.0,...,0,0,0,0,0,0,0,0,0,0
5,1679.0,20,1,51,0,0,1684.0,1803.0,1712.0,1714.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647364,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647365,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647366,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
5647367,NaN,18,7,231,0,13,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [19]:
def get_lag_features(df, subj_name):
    tmp = df[df["subject_name"] == subj_name].dropna()
    
    return tmp["actual_consumption"].iloc[-time_steps["hour"]], tmp["actual_consumption"].iloc[-time_steps["day"]], tmp["actual_consumption"].iloc[-time_steps["week"]], tmp["actual_consumption"].iloc[-time_steps["month"]], tmp["actual_consumption"].iloc[-time_steps["year"]]
        

In [20]:
cols_when_model_builds = model.get_booster().feature_names

In [21]:
main_df[:main_df_rows_count + 1]

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
1,1302.0,20,1,51,0,0,1284.0,1339.0,1367.0,1361.0,...,0,0,0,0,0,0,0,0,0,0
2,1091.0,20,1,51,0,0,1001.0,1034.0,1021.0,1116.0,...,0,0,0,0,0,0,0,0,0,0
3,946.0,20,1,51,0,0,959.0,982.0,926.0,996.0,...,0,0,0,0,0,0,0,0,0,0
4,615.0,20,1,51,0,0,617.0,638.0,535.0,644.0,...,0,0,0,0,0,0,0,0,0,0
5,1679.0,20,1,51,0,0,1684.0,1803.0,1712.0,1714.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647265,439.0,18,7,231,0,11,462.0,422.0,360.0,550.0,...,0,0,0,0,0,0,1,0,0,0
5647266,529.0,18,7,231,0,11,616.0,537.0,465.0,520.0,...,0,0,0,0,0,0,0,1,0,0
5647267,366.0,18,7,231,0,11,354.0,366.0,345.0,363.0,...,0,0,0,0,0,0,0,0,1,0
5647268,850.0,18,7,231,0,11,925.0,864.0,714.0,861.0,...,0,0,0,0,0,0,0,0,0,1


In [22]:
forecast_data_rows_count = 100

In [23]:
for idx in range(1, forecast_data_rows_count + 1):
# for idx in range(1, 10 + 1):
    lag_features = get_lag_features(main_df[:main_df_rows_count + idx - 1], 
                                    main_df.iloc[main_df_rows_count + idx - 1]["subject_name"])
    
#     main_df.at[main_df + idx, ["lag_hour", "lag_day", "lag_week", "lag_month", "lag_year"]] = lag_features
    
    main_df.at[main_df_rows_count + idx, "lag_hour"] = lag_features[0]
    main_df.at[main_df_rows_count + idx, "lag_day"] = lag_features[1]
    main_df.at[main_df_rows_count + idx, "lag_week"] = lag_features[2]
    main_df.at[main_df_rows_count + idx, "lag_month"] = lag_features[3]
    main_df.at[main_df_rows_count + idx, "lag_year"] = lag_features[4]

    main_df.at[main_df_rows_count + idx, "actual_consumption"] = model.predict(
        main_df[cols_when_model_builds].iloc[main_df_rows_count + idx - 1: main_df_rows_count + idx])

In [24]:
main_df[:main_df_rows_count + 2]

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
1,1302.000000,20,1,51,0,0,1284.0,1339.0,1367.0,1361.0,...,0,0,0,0,0,0,0,0,0,0
2,1091.000000,20,1,51,0,0,1001.0,1034.0,1021.0,1116.0,...,0,0,0,0,0,0,0,0,0,0
3,946.000000,20,1,51,0,0,959.0,982.0,926.0,996.0,...,0,0,0,0,0,0,0,0,0,0
4,615.000000,20,1,51,0,0,617.0,638.0,535.0,644.0,...,0,0,0,0,0,0,0,0,0,0
5,1679.000000,20,1,51,0,0,1684.0,1803.0,1712.0,1714.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647266,529.000000,18,7,231,0,11,616.0,537.0,465.0,520.0,...,0,0,0,0,0,0,0,1,0,0
5647267,366.000000,18,7,231,0,11,354.0,366.0,345.0,363.0,...,0,0,0,0,0,0,0,0,1,0
5647268,850.000000,18,7,231,0,11,925.0,864.0,714.0,861.0,...,0,0,0,0,0,0,0,0,0,1
5647269,1185.257324,18,7,231,0,12,1209.0,1191.0,1123.0,1168.0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
main_df[main_df["Ярославская область"] == 1]

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
71,1013.000000,20,1,51,0,0,982.0,1029.0,1000.0,1035.0,...,0,0,0,0,0,0,0,0,0,1
142,977.000000,20,1,51,0,1,980.0,1013.0,974.0,1031.0,...,0,0,0,0,0,0,0,0,0,1
213,978.000000,20,1,51,0,2,968.0,977.0,959.0,1034.0,...,0,0,0,0,0,0,0,0,0,1
284,986.000000,20,1,51,0,3,969.0,978.0,958.0,1036.0,...,0,0,0,0,0,0,0,0,0,1
355,1014.000000,20,1,51,0,4,949.0,986.0,974.0,1055.0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647043,831.000000,18,7,231,0,8,580.0,484.0,482.0,503.0,...,0,0,0,0,0,0,0,0,0,1
5647118,842.000000,18,7,231,0,9,583.0,509.0,474.0,510.0,...,0,0,0,0,0,0,0,0,0,1
5647193,864.000000,18,7,231,0,10,593.0,541.0,449.0,523.0,...,0,0,0,0,0,0,0,0,0,1
5647268,850.000000,18,7,231,0,11,925.0,864.0,714.0,861.0,...,0,0,0,0,0,0,0,0,0,1


In [26]:
ans = main_df[main_df_rows_count:]

In [27]:
ans.to_parquet("answer.gzip")

In [28]:
ans

,actual_consumption,day_of_month,day_of_week,day_of_year,holiday,hour,lag_day,lag_hour,lag_month,lag_week,...,Тульская область,Тюменская область,Удмуртская Республика,Ульяновская область,Центральный энергорайон Якутии,Челябинская область,Чеченская Республика,Чувашская Республика - Чувашия,Южно-Якутский энергорайон,Ярославская область
5647269,1185.257324,18,7,231,0,12,1209.0,1191.0,1123.0,1168.0,...,0,0,0,0,0,0,0,0,0,0
5647270,1065.297607,18,7,231,0,12,1070.0,1061.0,1060.0,1123.0,...,0,0,0,0,0,0,0,0,0,0
5647271,692.634827,18,7,231,0,12,712.0,687.0,589.0,689.0,...,0,0,0,0,0,0,0,0,0,0
5647272,487.566406,18,7,231,0,12,472.0,480.0,478.0,603.0,...,0,0,0,0,0,0,0,0,0,0
5647273,1804.980835,18,7,231,0,12,1777.0,1836.0,1470.0,1862.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5647364,796.097595,18,7,231,0,13,808.0,803.0,629.0,807.0,...,0,0,0,0,0,0,0,0,0,0
5647365,443.894745,18,7,231,0,13,440.0,445.0,310.0,446.0,...,0,0,0,0,0,0,0,0,0,0
5647366,4568.269043,18,7,231,0,13,4529.0,4516.0,3828.0,5333.0,...,0,0,0,0,0,0,0,0,0,0
5647367,5768.558594,18,7,231,0,13,5860.0,5797.0,5524.0,5821.0,...,0,0,0,0,0,0,0,0,0,0
